In [1]:
#importing libraries
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [2]:
#open the browser
driver=webdriver.Chrome()

#load the webpage
driver.get("https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile")
time.sleep(3)
driver.maximize_window()
wait_time = WebDriverWait(driver, 20)

def get_telangana_bus_data(xpath):
    bus_links = []  # Holds all the bus URLs
    bus_routes = []  # Holds all the route names
    
    for page_num in range(1, 5):
        route_elements = driver.find_elements(By.XPATH, xpath)
        bus_links.extend([element.get_attribute("href") for element in route_elements])
        bus_routes.extend([element.text for element in route_elements])
        
        try:
            pagination_table = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]'))
            )
            next_page_button = pagination_table.find_element(By.XPATH, f'//div[@class="DC_117_pageTabs " and text()="{page_num + 1}"]')
            
            # Scroll the next page button into view
            ActionChains(driver).move_to_element(next_page_button).perform()
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable(next_page_button))
            next_page_button.click()
            
        except NoSuchElementException:
            print(f"End of pages detected at page {page_num}")
            break
        except ElementClickInterceptedException:
            print(f"Could not click the next page button at page {page_num}, trying again.")
            time.sleep(1)  # Wait a moment before retrying
            next_page_button.click()
    
    return bus_links, bus_routes

In [3]:
# Run the function to get the bus links and routes
bus_links, bus_routes = get_telangana_bus_data("//a[@class='route']")

# Now create the DataFrame with the returned variables
telangana_df = pd.DataFrame({
    "Bus_Route_Name": bus_routes,  # variable name from ROUTE_TELANGANA
    "Bus_Route_Link": bus_links   # variable name from LINKS_TELANGANA
})

# Display the DataFrame
print(telangana_df)


End of pages detected at page 4
                             Bus_Route_Name  \
0                   Hyderabad to Vijayawada   
1                      Hyderabad to Khammam   
2                      Khammam to Hyderabad   
3                    Hyderabad to Srisailam   
4                   Hyderabad to Karimnagar   
5                   Hyderabad to Mancherial   
6                   Hyderabad to Sathupally   
7                     Hyderabad to Adilabad   
8                       Hyderabad to Nirmal   
9                 Hyderabad to Bhadrachalam   
10                  Karimnagar to Hyderabad   
11                      Hyderabad to Ongole   
12                  Hyderabad to Kothagudem   
13               Hyderabad to Visakhapatnam   
14     Hyderabad to Guntur (Andhra Pradesh)   
15                    Hyderabad to Tirupati   
16                  Kothagudem to Hyderabad   
17     Guntur (Andhra Pradesh) to Hyderabad   
18                    Hyderabad to Warangal   
19  Hyderabad to Anantapur (

In [9]:
# change dataframe to csv
path=r"C:\Users\Vanathi\Documents\redbus scraped details\telangana_df.csv"
telangana_df.to_csv(path,index=False)

In [5]:
df=pd.read_csv(r"C:\Users\Vanathi\Documents\redbus scraped details\telangana_df.csv")

In [6]:
driver_telangana = webdriver.Chrome()
Busname_1 = []
Bustype_1 = []
Starttime_1 = []
Endtime_1 = []
Ratings_1 = []
Total_duration_1 = []
Prices_1 = []
Seats_available_1 = []
Route_names = []
Route_links = []

for i, r in df.iterrows():
    link = r["Bus_Route_Link"]
    routes = r["Bus_Route_Name"]

    # Open the link
    driver_telangana.get(link)
    time.sleep(2)

    # Click on elements to reveal bus details
    elements = driver_telangana.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    # Click elements to view bus details
    try:
        clicks = driver_telangana.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue
    time.sleep(2)

    # Scroll through the page to load all bus details
    scrolling = True
    while scrolling:
        # Get the current scroll height
        last_height = driver_telangana.execute_script("return document.body.scrollHeight")

        # Scroll down to the bottom
        driver_telangana.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for new content to load
        time.sleep(5)

        # Get the new scroll height
        new_height = driver_telangana.execute_script("return document.body.scrollHeight")

        # Check if the page height has changed
        if new_height == last_height:
            scrolling = False

    # Extract bus details
    try:
        bus_name = WebDriverWait(driver_telangana, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='travels lh-24 f-bold d-color']"))
        )
        bus_type = driver_telangana.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        start_time = driver_telangana.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        end_time = driver_telangana.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver_telangana.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        try:
            rating = driver_telangana.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        except:
            continue
        price = driver_telangana.find_elements(By.XPATH, '//*[@class="fare d-block"]')
        seats = driver_telangana.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Append data to respective lists
        for bus in bus_name:
            Busname_1.append(bus.text)
            Route_links.append(link)
            Route_names.append(routes)
        for bus_type_elem in bus_type:
            Bustype_1.append(bus_type_elem.text)
        for start_time_elem in start_time:
            Starttime_1.append(start_time_elem.text)
        for end_time_elem in end_time:
            Endtime_1.append(end_time_elem.text)
        for total_duration_elem in total_duration:
            Total_duration_1.append(total_duration_elem.text)
        for ratings in rating:
            Ratings_1.append(ratings.text)
        for price_elem in price:
            Prices_1.append(price_elem.text)
        for seats_elem in seats:
            Seats_available_1.append(seats_elem.text)
    except Exception as e:
        print(f"Error occurred while extracting details: {e}")
        continue

print("Successfully Completed")



Successfully Completed


In [7]:
# Before converting to DataFrame, check if all lists have the same length
lengths = {
    'Busname_1': len(Busname_1),
    'Bustype_1': len(Bustype_1),
    'Starttime_1': len(Starttime_1),
    'Endtime_1': len(Endtime_1),
    'Total_duration_1': len(Total_duration_1),
    'Prices_1': len(Prices_1),
    'Seats_available_1': len(Seats_available_1),
    'Route_names': len(Route_names),
    'Route_links': len(Route_links),
    'Ratings_1': len(Ratings_1)
}

print(lengths)

# Ensure that all lists have the same length
max_length = max(lengths.values())  # Find the longest list
for lst in [Busname_1, Bustype_1, Starttime_1, Endtime_1, Total_duration_1, Prices_1, Seats_available_1, Route_names, Route_links, Ratings_1]:
    while len(lst) < max_length:
        lst.append(None)  # Or you can append a placeholder like an empty string


{'Busname_1': 1472, 'Bustype_1': 1472, 'Starttime_1': 1472, 'Endtime_1': 1472, 'Total_duration_1': 1472, 'Prices_1': 1472, 'Seats_available_1': 1472, 'Route_names': 1472, 'Route_links': 1472, 'Ratings_1': 201}


In [11]:
# Convert list to DataFrame
data = {
    'Bus_name': Busname_1,
    'Bus_type': Bustype_1,
    'Start_time': Starttime_1,
    'End_time': Endtime_1,
    'Total_duration': Total_duration_1,
    'Price': Prices_1,
    "Seats_Available": Seats_available_1,
    "Ratings": Ratings_1,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_3= pd.DataFrame(data)

# Convert DataFrame to CSV
path = r"C:\Users\Vanathi\Documents\redbus scraped details\df_buses_3.csv"
df_buses_3.to_csv(path, index=False)

print("Successfully Completed and saved to CSV.")

Successfully Completed and saved to CSV.


In [12]:
c=pd.read_csv(r"C:\Users\Vanathi\Documents\redbus scraped details\df_buses_3.csv")
c

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,APSRTC - 35188,VENNELA (A.C. SLEEPER),00:40,06:45,06h 05m,INR 737,16 Seats available,4.7\n9,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
1,APSRTC - 3563,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",01:00,06:35,05h 35m,INR 412,32 Seats available,3.6,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
2,APSRTC - 35266,VENNELA (A.C. SLEEPER),01:25,07:10,05h 45m,INR 737,19 Seats available,3.6,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
3,APSRTC - 3590,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",01:45,08:00,06h 15m,INR 412,32 Seats available,4.1\n15,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
4,APSRTC - 3058,VENNELA (A.C. SLEEPER),02:10,08:15,06h 05m,INR 737,21 Seats available,3.5\n27,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
...,...,...,...,...,...,...,...,...,...,...
1467,TGSRTC - 7192,Super Luxury (Non AC Seater 2+2 Push Back),23:30,04:20,04h 50m,INR 392,25 Seats available,NaN,https://www.redbus.in/bus-tickets/godavarikhan...,Godavarikhani to Hyderabad
1468,TGSRTC - 9508,Ordinary Non AC Seater 2+3,23:45,04:20,04h 35m,INR 476,31 Seats available,NaN,https://www.redbus.in/bus-tickets/godavarikhan...,Godavarikhani to Hyderabad
1469,Orange Tours and Travels,Scania AC Multi Axle Sleeper (2+1),17:30,22:05,04h 35m,INR 600,18 Seats available,NaN,https://www.redbus.in/bus-tickets/godavarikhan...,Godavarikhani to Hyderabad
1470,Sri Krishna Travels,Volvo 9600 Multi-Axle A/C Sleeper (2+1),17:15,22:35,05h 20m,INR 5000,42 Seats available,NaN,https://www.redbus.in/bus-tickets/godavarikhan...,Godavarikhani to Hyderabad
